<a href="https://colab.research.google.com/github/abhaydubey200/Second_Project-_Price_prediction_model/blob/main/Prophet_RF_Data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd

df = pd.read_csv("/content/drive/MyDrive/DS(Data)/final_master_dataset.csv")

df.head()

,Country,Year_x,Month_x,year_month,production,Year_y,Month_y,Rainfall,Date of Auction,Quantity,Price,Date,Month,Month_Num,Year
0,India,2020,1,2020-01,22.5,2020.0,Jan,5.9,2020-01-31,9700.7,3657.52,2020-01-31,1.0,1.0,2020.0
1,India,2020,1,2020-01,22.5,2020.0,Jan,5.9,2020-01-31,44579.1,3597.85,2020-01-31,1.0,1.0,2020.0
2,India,2020,1,2020-01,22.5,2020.0,Jan,5.9,2020-01-30,20245.8,3629.36,2020-01-30,1.0,1.0,2020.0
3,India,2020,1,2020-01,22.5,2020.0,Jan,5.9,2020-01-30,39248.2,3685.66,2020-01-30,1.0,1.0,2020.0
4,India,2020,1,2020-01,22.5,2020.0,Jan,5.9,2020-01-29,11680.5,3655.21,2020-01-29,1.0,1.0,2020.0


In [ ]:
df.columns

Index(['Country', 'Year_x', 'Month_x', 'year_month', 'production', 'Year_y',
       'Month_y', 'Rainfall', 'Date of Auction', 'Quantity', 'Price', 'Date',
       'Month', 'Month_Num', 'Year'],
      dtype='object')

In [ ]:
df.isnull().sum()

,0
Country,0
Year_x,0
Month_x,0
year_month,0
production,0
Year_y,1512
Month_y,1512
Rainfall,1512
Date of Auction,18
Quantity,18


In [ ]:
df = df.dropna(subset=['Price', 'Quantity'])

In [ ]:
df['Rainfall'] = df['Rainfall'].interpolate()

/tmp/ipython-input-2047134149.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Rainfall'] = df['Rainfall'].interpolate()


In [ ]:
df = df.drop(columns=['Year_x', 'Year_y', 'Month_x', 'Month_y'], errors='ignore')

In [ ]:
df['ds'] = pd.to_datetime(df['Date'])
df = df.sort_values('ds')

In [ ]:
print(df.isna().sum())

Country            0
year_month         0
production         0
Rainfall           0
Date of Auction    0
Quantity           0
Price              0
Date               0
Month              0
Month_Num          0
Year               0
ds                 0
dtype: int64


In [ ]:
# pip install prophet

In [ ]:
from prophet import Prophet

price_df = df[['ds', 'Price']].rename(columns={'Price': 'y'})
model_price = Prophet()
model_price.fit(price_df)
future_price = model_price.make_future_dataframe(periods=12, freq='M')
forecast_price = model_price.predict(future_price)

INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
/usr/local/lib/python3.12/dist-packages/prophet/forecaster.py:1872: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  dates = pd.date_range(


In [ ]:
quantity_df = df[['ds', 'Quantity']].rename(columns={'Quantity': 'y'})
model_quantity = Prophet()
model_quantity.fit(quantity_df)
future_quantity = model_quantity.make_future_dataframe(periods=12, freq='M')
forecast_quantity = model_quantity.predict(future_quantity)

INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
/usr/local/lib/python3.12/dist-packages/prophet/forecaster.py:1872: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  dates = pd.date_range(


In [ ]:
final_prophet = pd.DataFrame()
final_prophet['Month'] = forecast_price['ds']
final_prophet['Predicted_Price'] = forecast_price['yhat']
final_prophet['Predicted_Quantity'] = forecast_quantity['yhat']

final_prophet.to_csv("Prophet_Tableau_Output.csv", index=False)

In [ ]:
future_price = model_price.make_future_dataframe(periods=12, freq='M')
forecast_price = model_price.predict(future_price)
forecast_price.head()

/usr/local/lib/python3.12/dist-packages/prophet/forecaster.py:1872: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  dates = pd.date_range(


,ds,trend,yhat_lower,yhat_upper,trend_lower,trend_upper,additive_terms,additive_terms_lower,additive_terms_upper,weekly,weekly_lower,weekly_upper,yearly,yearly_lower,yearly_upper,multiplicative_terms,multiplicative_terms_lower,multiplicative_terms_upper,yhat
0,2020-01-01,3599.418290,3916.084244,4162.573251,3599.418290,3599.418290,445.491191,445.491191,445.491191,-12.134556,-12.134556,-12.134556,457.625747,457.625747,457.625747,0.0,0.0,0.0,4044.909481
1,2020-01-02,3583.235218,3906.846283,4159.055301,3583.235218,3583.235218,451.896044,451.896044,451.896044,-7.043634,-7.043634,-7.043634,458.939678,458.939678,458.939678,0.0,0.0,0.0,4035.131263
2,2020-01-03,3567.052147,3899.055306,4157.596214,3567.052147,3567.052147,458.805710,458.805710,458.805710,-1.114223,-1.114223,-1.114223,459.919933,459.919933,459.919933,0.0,0.0,0.0,4025.857857
3,2020-01-04,3550.869076,3894.754436,4134.537758,3550.869076,3550.869076,465.397131,465.397131,465.397131,4.826645,4.826645,4.826645,460.570486,460.570486,460.570486,0.0,0.0,0.0,4016.266207
4,2020-01-06,3518.502933,3849.589672,4099.603782,3518.502933,3518.502933,459.446646,459.446646,459.446646,-1.464301,-1.464301,-1.464301,460.910947,460.910947,460.910947,0.0,0.0,0.0,3977.949579


from matplotlib import pyplot as plt
_df_0['trend'].plot(kind='hist', bins=20, title='trend')
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
_df_1['yhat_lower'].plot(kind='hist', bins=20, title='yhat_lower')
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
_df_2['yhat_upper'].plot(kind='hist', bins=20, title='yhat_upper')
plt.gca().spines[['top', 'right',]].set_visible(False)

...

from matplotlib import pyplot as plt
import seaborn as sns
def _plot_series(series, series_name, series_index=0):
  palette = list(sns.palettes.mpl_palette('Dark2'))
  xs = series['ds']
  ys = series['trend_lower']
  
  plt.plot(xs, ys, label=series_name, color=palette[series_index % len(palette)])

fig, ax = plt.subplots(figsize=(10, 5.2), layout='constrained')
df_sorted = _df_11.sort_values('ds', ascending=True)
_plot_series(df_sorted, '')
sns.despine(fig=fig, ax=ax)
plt.xlabel('ds')
_ = plt.ylabel('trend_lower')

from matplotlib import pyplot as plt
_df_12['trend'].plot(kind='line', figsize=(8, 4), title='trend')
plt.gca().spines[['top', 'right']].set_visible(False)

from matplotlib import pyplot as plt
_df_13['yhat_lower'].plot(kind='line', figsize=(8, 4), title='yhat_lower')
plt.gca().spines[['top', 'right']].set_visible(False)

from matplotlib import pyplot as plt
_df_14['yhat_upper'].plot(kind='line', figsize=(8, 4), title='yhat_upper')
plt.gca().spines[['top', 'right']].set_visible(False)

from matplotlib import pyplot as plt
_df_15['trend_lower'].plot(kind='line', figsize=(8, 4), title='trend_lower')
plt.gca().spines[['top', 'right']].set_visible(False)

In [ ]:
quantity_df = df[['ds', 'Quantity']].rename(columns={'Quantity': 'y'})
quantity_df.head()

,ds,y
1114,2020-01-01,61562.8
1113,2020-01-01,36141.8
759,2020-01-01,61562.8
758,2020-01-01,36141.8
1468,2020-01-01,36141.8


In [ ]:
model_quantity = Prophet()
model_quantity.fit(quantity_df)

INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


In [ ]:
future_quantity = model_quantity.make_future_dataframe(periods=12, freq='M')
forecast_quantity = model_quantity.predict(future_quantity)

/usr/local/lib/python3.12/dist-packages/prophet/forecaster.py:1872: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  dates = pd.date_range(


In [ ]:
final_prophet = pd.DataFrame()
final_prophet['Month'] = forecast_price['ds']
final_prophet['Predicted_Price'] = forecast_price['yhat']
final_prophet['Predicted_Quantity'] = forecast_quantity['yhat']

In [ ]:
final_prophet.to_csv("Prophet_Tableau_Output.csv", index=False)
print("Prophet forecast saved → Prophet_Tableau_Output.csv")

Prophet forecast saved → Prophet_Tableau_Output.csv


In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split


df['Price_lag1'] = df['Price'].shift(1)
df['Price_lag2'] = df['Price'].shift(2)
df['Price_lag3'] = df['Price'].shift(3)

df['Quantity_lag1'] = df['Quantity'].shift(1)
df['Quantity_lag2'] = df['Quantity'].shift(2)
df['Quantity_lag3'] = df['Quantity'].shift(3)

df['production_lag1'] = df['production'].shift(1)
df['production_lag2'] = df['production'].shift(2)
df['production_lag3'] = df['production'].shift(3)

df['Rainfall_lag1'] = df['Rainfall'].shift(1)
df['Rainfall_lag2'] = df['Rainfall'].shift(2)
df['Rainfall_lag3'] = df['Rainfall'].shift(3)

df['Price_MA3'] = df['Price'].rolling(window=3).mean()
df['Price_MA6'] = df['Price'].rolling(window=6).mean()

df_rf = df.dropna(subset=['Price_lag3', 'Quantity_lag3', 'production_lag3', 'Rainfall_lag3', 'Price_MA6']).copy()

features = [
    'Rainfall','Quantity','production',
    'Price_lag1','Price_lag2','Price_lag3',
    'Quantity_lag1','Quantity_lag2','Quantity_lag3',
    'production_lag1','production_lag2','production_lag3',
    'Rainfall_lag1','Rainfall_lag2','Rainfall_lag3',
    'Price_MA3','Price_MA6'
]

X = df_rf[features]
y = df_rf['Price']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)

model_rf = RandomForestRegressor(n_estimators=300, random_state=42)
model_rf.fit(X_train, y_train)

df_rf['RF_Predicted'] = model_rf.predict(X)



In [ ]:
df_rf.to_csv("RF_Tableau_Output.csv", index=False)
df_rf[['Date','Price','RF_Predicted']].head(20)

,Date,Price,RF_Predicted
1469,2020-01-01,3755.26,3755.013133
403,2020-01-01,3756.37,3756.839967
1823,2020-01-01,3756.37,3758.089800
49,2020-01-01,3755.26,3755.140133
48,2020-01-01,3756.37,3756.044833
404,2020-01-01,3755.26,3753.983800
1824,2020-01-01,3755.26,3753.831267
1112,2020-01-02,3802.22,3800.655333
1467,2020-01-02,3802.22,3804.115600
1111,2020-01-02,3696.14,3705.613900


In [ ]:
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split


df = pd.read_csv("/content/drive/MyDrive/DS(Data)/final_master_dataset.csv")

df = df.dropna(subset=['Price', 'Quantity'])

df['Rainfall'] = df['Rainfall'].interpolate()
df['Rainfall'] = df['Rainfall'].bfill().ffill()

df = df.drop(columns=['Year_x','Year_y','Month_x','Month_y'], errors='ignore')

if 'Date' in df.columns:
    df['Date'] = pd.to_datetime(df['Date'])
    df = df.sort_values('Date')

df['Price_lag1'] = df['Price'].shift(1)
df['Price_lag2'] = df['Price'].shift(2)
df['Price_lag3'] = df['Price'].shift(3)

df['Quantity_lag1'] = df['Quantity'].shift(1)
df['Quantity_lag2'] = df['Quantity'].shift(2)
df['Quantity_lag3'] = df['Quantity'].shift(3)

df['production_lag1'] = df['production'].shift(1)
df['production_lag2'] = df['production'].shift(2)
df['production_lag3'] = df['production'].shift(3)

df['Rainfall_lag1'] = df['Rainfall'].shift(1)
df['Rainfall_lag2'] = df['Rainfall'].shift(2)
df['Rainfall_lag3'] = df['Rainfall'].shift(3)

df['Price_MA3'] = df['Price'].rolling(window=3).mean()
df['Price_MA6'] = df['Price'].rolling(window=6).mean()
df_processed = df.dropna(subset=[
    'Price_lag3', 'Quantity_lag3', 'production_lag3', 'Rainfall_lag3', 'Price_MA6'
]).copy()

features = [
    'Rainfall','Quantity','production',
    'Price_lag1','Price_lag2','Price_lag3',
    'Quantity_lag1','Quantity_lag2','Quantity_lag3',
    'production_lag1','production_lag2','production_lag3',
    'Rainfall_lag1','Rainfall_lag2','Rainfall_lag3',
    'Price_MA3','Price_MA6'
]

X = df_processed[features]

y_price = df_processed['Price']

X_train, X_test, y_price_train, y_price_test = train_test_split(
    X, y_price, test_size=0.2, shuffle=False
)

rf_price = RandomForestRegressor(n_estimators=300, random_state=42)
rf_price.fit(X_train, y_price_train)

df_processed['RF_Predicted_Price'] = rf_price.predict(X)

y_qty = df_processed['Quantity']

y_qty_train = y_qty.loc[X_train.index]
y_qty_test = y_qty.loc[X_test.index]

rf_qty = RandomForestRegressor(n_estimators=300, random_state=42)
rf_qty.fit(X_train, y_qty_train)

df_processed['RF_Predicted_Quantity'] = rf_qty.predict(X)

df_processed[['Date', 'Price', 'RF_Predicted_Price', 'Quantity', 'RF_Predicted_Quantity']].head(20)


,Date,Price,RF_Predicted_Price,Quantity,RF_Predicted_Quantity
1469,2020-01-01,3755.26,3755.013133,61562.8,61562.779000
403,2020-01-01,3756.37,3756.839967,36141.8,36142.792000
1823,2020-01-01,3756.37,3758.089800,36141.8,36142.890333
49,2020-01-01,3755.26,3755.140133,61562.8,61562.779000
48,2020-01-01,3756.37,3756.044833,36141.8,36142.031333
404,2020-01-01,3755.26,3753.983800,61562.8,61562.713000
1824,2020-01-01,3755.26,3753.831267,61562.8,61562.739000
1112,2020-01-02,3802.22,3800.655333,74318.7,74318.609333
1467,2020-01-02,3802.22,3804.115600,74318.7,74318.974667
1111,2020-01-02,3696.14,3705.613900,15314.4,15314.489333


In [ ]:

df_processed.to_csv("/content/drive/MyDrive/DS(Data)/final_master_dataset_with_predictions.csv", index=False)
print(" Processed dataset with predictions saved successfully!")

✅ Processed dataset with predictions saved successfully!


In [1]:
import pandas as pd
from xgboost import XGBRegressor
from sklearn.model_selection import train_test_split

df = pd.read_csv("/content/drive/MyDrive/DS(Data)/final_master_dataset.csv")

df = df.dropna(subset=['Price', 'Quantity'])

df['Rainfall'] = df['Rainfall'].interpolate()
df['Rainfall'] = df['Rainfall'].bfill().ffill()

df = df.drop(columns=['Year_x','Year_y','Month_x','Month_y'], errors='ignore')

if 'Date' in df.columns:
    df['Date'] = pd.to_datetime(df['Date'])
    df = df.sort_values('Date')

df['Price_lag1'] = df['Price'].shift(1)
df['Price_lag2'] = df['Price'].shift(2)
df['Price_lag3'] = df['Price'].shift(3)

df['Quantity_lag1'] = df['Quantity'].shift(1)
df['Quantity_lag2'] = df['Quantity'].shift(2)
df['Quantity_lag3'] = df['Quantity'].shift(3)

df['production_lag1'] = df['production'].shift(1)
df['production_lag2'] = df['production'].shift(2)
df['production_lag3'] = df['production'].shift(3)

df['Rainfall_lag1'] = df['Rainfall'].shift(1)
df['Rainfall_lag2'] = df['Rainfall'].shift(2)
df['Rainfall_lag3'] = df['Rainfall'].shift(3)

df['Price_MA3'] = df['Price'].rolling(window=3).mean()
df['Price_MA6'] = df['Price'].rolling(window=6).mean()

df_processed = df.dropna(subset=[
    'Price_lag3','Quantity_lag3','production_lag3','Rainfall_lag3','Price_MA6'
]).copy()

features = [
    'Rainfall','Quantity','production',
    'Price_lag1','Price_lag2','Price_lag3',
    'Quantity_lag1','Quantity_lag2','Quantity_lag3',
    'production_lag1','production_lag2','production_lag3',
    'Rainfall_lag1','Rainfall_lag2','Rainfall_lag3',
    'Price_MA3','Price_MA6'
]

X = df_processed[features]

y_price = df_processed['Price']

X_train, X_test, y_price_train, y_price_test = train_test_split(
    X, y_price, test_size=0.2, shuffle=False
)

xgb_price = XGBRegressor(
    n_estimators=300,
    learning_rate=0.05,
    max_depth=6,
    subsample=0.9,
    colsample_bytree=0.9,
    random_state=42
)

xgb_price.fit(X_train, y_price_train)

df_processed['XGB_Predicted_Price'] = xgb_price.predict(X)

y_qty = df_processed['Quantity']

y_qty_train = y_qty.loc[X_train.index]
y_qty_test = y_qty.loc[X_test.index]

xgb_qty = XGBRegressor(
    n_estimators=300,
    learning_rate=0.05,
    max_depth=6,
    subsample=0.9,
    colsample_bytree=0.9,
    random_state=42
)

xgb_qty.fit(X_train, y_qty_train)

df_processed['XGB_Predicted_Quantity'] = xgb_qty.predict(X)

df_processed[['Date','Price','XGB_Predicted_Price','Quantity','XGB_Predicted_Quantity']].head(20)

,Date,Price,XGB_Predicted_Price,Quantity,XGB_Predicted_Quantity
1469,2020-01-01,3755.26,3761.575684,61562.8,61558.855469
403,2020-01-01,3756.37,3764.544678,36141.8,36861.058594
1823,2020-01-01,3756.37,3762.700684,36141.8,36584.476562
49,2020-01-01,3755.26,3762.893311,61562.8,61570.328125
48,2020-01-01,3756.37,3765.934570,36141.8,36226.117188
404,2020-01-01,3755.26,3762.122559,61562.8,61791.031250
1824,2020-01-01,3755.26,3761.836182,61562.8,61576.539062
1112,2020-01-02,3802.22,3809.789307,74318.7,74224.015625
1467,2020-01-02,3802.22,3791.716064,74318.7,74397.187500
1111,2020-01-02,3696.14,3707.850342,15314.4,14704.524414


In [2]:
df_processed.to_csv("/content/drive/MyDrive/DS(Data)/final_master_dataset_with_predictions.csv",
                    index=False)

print("Processed dataset with predictions saved successfully!")

✅ Processed dataset with predictions saved successfully!


In [3]:
!pip install tensorflow

In [5]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.callbacks import EarlyStopping

df = pd.read_csv("/content/drive/MyDrive/DS(Data)/final_master_dataset.csv")

cols = ["Price", "Quantity"]
df = df[cols]

df = df.dropna().reset_index(drop=True)

scaler = MinMaxScaler()
scaled_data = scaler.fit_transform(df)

scaled_df = pd.DataFrame(scaled_data, columns=["Price", "Quantity"])

def create_sequences(data, seq_len=30):
    X, y_price, y_qty = [], [], []

    for i in range(seq_len, len(data)):
        X.append(data[i-seq_len:i])
        y_price.append(data[i, 0])
        y_qty.append(data[i, 1])

    return np.array(X), np.array(y_price), np.array(y_qty)

SEQ_LEN = 30
X, y_price, y_qty = create_sequences(scaled_df.values, SEQ_LEN)

train_size = int(0.8 * len(X))
X_train, X_test = X[:train_size], X[train_size:]
y_price_train, y_price_test = y_price[:train_size], y_price[train_size:]
y_qty_train, y_qty_test = y_qty[:train_size], y_qty[train_size:]

model = Sequential([
    LSTM(64, return_sequences=True, input_shape=(SEQ_LEN, 2)),
    LSTM(32),
    Dense(2)
])

model.compile(optimizer="adam", loss="mse")

model.fit(
    X_train, np.column_stack([y_price_train, y_qty_train]),
    epochs=25,
    batch_size=32,
    validation_split=0.1,
    callbacks=[EarlyStopping(monitor="loss", patience=5, restore_best_weights=True)]
)

pred_scaled = model.predict(X)

pred_original = scaler.inverse_transform(pred_scaled)

df_pred = df.copy()
df_pred["LSTM_Predicted_Price"] = np.nan
df_pred["LSTM_Predicted_Quantity"] = np.nan

df_pred.loc[SEQ_LEN:, "LSTM_Predicted_Price"] = pred_original[:, 0]
df_pred.loc[SEQ_LEN:, "LSTM_Predicted_Quantity"] = pred_original[:, 1]

df_pred.head(20)

/usr/local/lib/python3.12/dist-packages/keras/src/layers/rnn/rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/25
399/399 ━━━━━━━━━━━━━━━━━━━━ 16s 32ms/step - loss: 0.0191 - val_loss: 0.0125
Epoch 2/25
399/399 ━━━━━━━━━━━━━━━━━━━━ 13s 31ms/step - loss: 0.0121 - val_loss: 0.0123
Epoch 3/25
399/399 ━━━━━━━━━━━━━━━━━━━━ 20s 31ms/step - loss: 0.0120 - val_loss: 0.0121
Epoch 4/25
399/399 ━━━━━━━━━━━━━━━━━━━━ 12s 31ms/step - loss: 0.0119 - val_loss: 0.0120
Epoch 5/25
399/399 ━━━━━━━━━━━━━━━━━━━━ 13s 31ms/step - loss: 0.0116 - val_loss: 0.0116
Epoch 6/25
399/399 ━━━━━━━━━━━━━━━━━━━━ 12s 31ms/step - loss: 0.0111 - val_loss: 0.0115
Epoch 7/25
399/399 ━━━━━━━━━━━━━━━━━━━━ 12s 31ms/step - loss: 0.0109 - val_loss: 0.0117
Epoch 8/25
399/399 ━━━━━━━━━━━━━━━━━━━━ 11s 27ms/step - loss: 0.0109 - val_loss: 0.0116
Epoch 9/25
399/399 ━━━━━━━━━━━━━━━━━━━━ 12s 30ms/step - loss: 0.0111 - val_loss: 0.0113
Epoch 10/25
399/399 ━━━━━━━━━━━━━━━━━━━━ 12s 31ms/step - loss: 0.0110 - val_loss: 0.0113
Epoch 11/25
399/399 ━━━━━━━━━━━━━━━━━━━━ 20s 31ms/step - loss: 0.0107 - val_loss: 0.0117
Epoch 12/25
399/399 ━━━━━━━━━━

,Price,Quantity,LSTM_Predicted_Price,LSTM_Predicted_Quantity
0,3657.52,9700.7,NaN,NaN
1,3597.85,44579.1,NaN,NaN
2,3629.36,20245.8,NaN,NaN
3,3685.66,39248.2,NaN,NaN
4,3655.21,11680.5,NaN,NaN
5,3665.92,10892.5,NaN,NaN
6,3706.02,8321.7,NaN,NaN
7,3665.44,32594.9,NaN,NaN
8,3720.98,28982.4,NaN,NaN
9,3731.52,34676.4,NaN,NaN


In [6]:

df_pred.tail(20)

,Price,Quantity,LSTM_Predicted_Price,LSTM_Predicted_Quantity
17704,2532.54,111465.6,2122.350830,65835.890625
17705,2453.12,36131.8,2118.649414,22303.625000
17706,2482.35,72993.8,2243.227295,102491.710938
17707,2468.85,79655.6,2719.350586,64347.566406
17708,2434.57,35791.1,2339.391602,55470.117188
17709,2423.78,86313.4,2728.472656,67689.726562
17710,2392.60,94665.2,2813.684082,58957.980469
17711,2416.05,68822.0,2279.882080,75086.890625
17712,2431.33,103276.9,2655.588867,70793.781250
17713,2362.00,17490.5,2156.581787,51988.597656


In [8]:

df_final = df_pred.copy().iloc[30:].reset_index(drop=True)

df_final.to_csv("/content/drive/MyDrive/DS(Data)/final_master_dataset_LSTM_cleaned.csv", index=False)

df_final.head()

,Price,Quantity,LSTM_Predicted_Price,LSTM_Predicted_Quantity
0,3827.18,54418.9,3502.358398,27012.826172
1,3811.61,63889.9,3896.276123,41811.042969
2,3859.47,72398.9,3628.374512,40642.308594
3,3801.64,22939.1,3755.620361,21636.843750
4,3880.86,81764.7,3857.702637,60078.230469


In [13]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from math import sqrt

# Prophet
from prophet import Prophet

# LSTM
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from sklearn.preprocessing import MinMaxScaler

df = pd.read_csv("/content/drive/MyDrive/DS(Data)/final_master_dataset.csv")
df["Date"] = pd.to_datetime(df["Date"])
# Drop rows where 'Date' is NaN before proceeding with Prophet
df = df.dropna(subset=['Date'])
df = df.sort_values("Date")

features = ["Quantity", "Rainfall"]
target = "Price"

X = df[features]
y = df[target]


split = int(len(df) * 0.8)
X_train, X_test = X[:split], X[split:]
y_train, y_test = y[:split], y[split:]

#  RANDOM FOREST

rf = RandomForestRegressor(n_estimators=300, random_state=42)
rf.fit(X_train, y_train)
rf_pred = rf.predict(X_test)

#  XGBOOST
xgb = XGBRegressor(n_estimators=300, learning_rate=0.05, max_depth=5)
xgb.fit(X_train, y_train)
xgb_pred = xgb.predict(X_test)

#  PROPHET
prophet_df = df[["Date", "Price"]].rename(columns={"Date": "ds", "Price": "y"})
train_prophet = prophet_df.iloc[:split]
test_prophet = prophet_df.iloc[split:]

model_p = Prophet()
model_p.fit(train_prophet)

# Predict directly on the dates from the test set to avoid out-of-bounds errors
forecast = model_p.predict(pd.DataFrame({'ds': test_prophet['ds']}))
prophet_pred = forecast["yhat"].values

#  LSTM
# Apply the same cleaning as previous steps to ensure no NaNs
df_lstm_cleaned = df.dropna(subset=['Price', 'Quantity']).copy()
df_lstm_cleaned['Rainfall'] = df_lstm_cleaned['Rainfall'].interpolate(limit_direction='both').ffill().bfill()
lstm_df = df_lstm_cleaned[["Price", "Quantity", "Rainfall"]]

scaler = MinMaxScaler()
scaled = scaler.fit_transform(lstm_df)

lookback = 12

def create_data(data):
    X, y = [], []
    for i in range(lookback, len(data)):
        X.append(data[i-lookback:i, :])
        y.append(data[i, 0])   # Price
    return np.array(X), np.array(y)

X_lstm, y_lstm = create_data(scaled)

# Adjust split point for LSTM data as it starts after 'lookback' items
# The number of samples for LSTM is len(X_lstm), so the split needs to be relative to that.
# Calculate the actual split point for the LSTM data based on its length
lstm_split_point = int(len(X_lstm) * 0.8)

X_train_lstm, X_test_lstm = X_lstm[:lstm_split_point], X_lstm[lstm_split_point:]
y_train_lstm, y_test_lstm = y_lstm[:lstm_split_point], y_lstm[lstm_split_point:]

model_lstm = Sequential([
    LSTM(64, return_sequences=True, input_shape=(lookback, 3)),
    LSTM(32),
    Dense(1)
])

model_lstm.compile(optimizer='adam', loss='mse')
model_lstm.fit(X_train_lstm, y_train_lstm, epochs=20, batch_size=16, verbose=0)

lstm_pred_scaled = model_lstm.predict(X_test_lstm)
lstm_pred = scaler.inverse_transform(
    np.concatenate([lstm_pred_scaled, np.zeros((len(lstm_pred_scaled), 2))], axis=1)
)[:, 0]

y_test_lstm_actual = scaler.inverse_transform(
    np.concatenate([y_test_lstm.reshape(-1,1), np.zeros((len(y_test_lstm),2))], axis=1)
)[:,0]

# ACCURACY FUNCTIONS
def evaluate(actual, pred):
    mae = mean_absolute_error(actual, pred)
    rmse = sqrt(mean_squared_error(actual, pred))
    mape = np.mean(np.abs((actual - pred) / actual)) * 100
    return mae, rmse, mape

# Evaluate all
rf_scores = evaluate(y_test, rf_pred)
xgb_scores = evaluate(y_test, xgb_pred)
prophet_scores = evaluate(y_test.values, prophet_pred)
lstm_scores = evaluate(y_test_lstm_actual, lstm_pred)

# PRINT RESULTS
results = pd.DataFrame({
    "Model": ["Random Forest", "XGBoost", "Prophet", "LSTM"],
    "MAE": [rf_scores[0], xgb_scores[0], prophet_scores[0], lstm_scores[0]],
    "RMSE": [rf_scores[1], xgb_scores[1], prophet_scores[1], lstm_scores[1]],
    "MAPE (%)": [rf_scores[2], xgb_scores[2], prophet_scores[2], lstm_scores[2]]
})

print("========== TEST ACCURACY (LOWER = BETTER) ==========\n")
print(results)


INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
/usr/local/lib/python3.12/dist-packages/keras/src/layers/rnn/rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


111/111 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step
========== TEST ACCURACY (LOWER = BETTER) ==========

           Model          MAE         RMSE   MAPE (%)
0  Random Forest  1218.854789  1305.319729  46.695834
1        XGBoost  1033.838831  1144.976216  39.521504
2        Prophet   463.162790   556.107537  17.036270
3           LSTM    50.527157    59.078781   1.925114


# 1. Test Accuracy Overview (Lower = Better)

| Model         | MAE     | RMSE    | MAPE (%) |
| ------------- | ------- | ------- | -------- |
| Random Forest | 1218.85 | 1305.32 | 46.70    |
| XGBoost       | 1033.84 | 1144.98 | 39.52    |
| Prophet       | 463.16  | 556.11  | 17.04    |
| LSTM          | 50.53   | 59.08   | 1.93     |


# 2.Interpretation of Results

Random Forest:

High error (MAE 1218, MAPE 46.7%)

Struggles with time-dependency; treats each row independently

Cannot model monthly seasonality or market cycles

XGBoost:

Better than Random Forest (MAE 1033, MAPE 39.5%)

Captures some non-linear relationships

Still weak for sequential/time-series data

Prophet:

MAE 463, MAPE 17.0%

Handles yearly seasonality, trends, and market cycles

Good for medium-term monthly forecasting

LSTM Neural Network:

MAE 50.5, MAPE 1.93% → production-level accuracy

Learns long-term sequential dependencies

Captures chain effect: Rainfall → Quantity → Price

Best model for time-series data with seasonal cycles

# 3. Ranking of Models

| Rank | Model         | Summary                                                 |
| ---- | ------------- | ------------------------------------------------------- |
|  #1 | LSTM          | Best model; captures sequential dependencies accurately |
|  #2 | Prophet       | Good for trend & seasonality; moderate accuracy         |
|  #3 | XGBoost       | Non-linear relations captured but fails time dependency |
|  #4 | Random Forest | Overfits; cannot capture temporal patterns              |


# 4️. Why LSTM is the Best for This Dataset

Cardamom prices are time-series dependent:

Past prices influence current prices

Rainfall impacts quantity → Quantity impacts price

Seasonal cycles repeat annually

LSTM excels at learning long-term sequential patterns, making it ideal for forecasting:

Learns month-by-month dependencies

Can predict future prices based on past 12 months of data

Achieves MAPE 1.93%, suitable for real-world business use.

# 5️. Why Prophet Performs Well

Automatically captures seasonality, trends, and market cycles

Works well for monthly forecasting

Handles periods of high price fluctuations (e.g., festivals, low production)

# 6️. Why Tree-Based Models Underperform

Random Forest and XGBoost do not inherently understand time-series data

Treats every row as independent → misses sequential patterns

Struggles with non-stationary, volatile datasets like cardamom prices

# 7️. Recommended Summary for DS Group Report

“Among the four models tested, the LSTM Neural Network achieved the highest accuracy with a MAPE of 1.93%.
LSTM effectively learns sequential dependencies, such as the impact of rainfall on quantity and quantity on price.
Prophet performed moderately well due to its ability to capture trends and seasonality.
Tree-based models (Random Forest and XGBoost) underperformed because they do not inherently model temporal dependencies, and the dataset is highly non-stationary.”

# 8️. Suggestion

Model Performance Comparison – Key Insights:

LSTM (Best Model): Captures long-term market cycles → Highest accuracy

Prophet: Captures trend & seasonality → Moderate accuracy

XGBoost: Captures non-linear relationships → Weak on sequential patterns

Random Forest: Highest error → Not suitable for time-series forecasting